# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json


import seaborn as sns
import scipy.stats as stats
from census import Census
from IPython.display import display
import ipywidgets as widgets
import gmaps.geojson_geometries


# Import API key
from config import g_key

ImportError: cannot import name 'g_key' from 'config' (C:\Users\arnol\Desktop\SMU-Data-Science-Homework\Homework Submissions\Python API Challenge\config.py)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
filepath = "../output_data/cities.csv"
base_df = pd.read_csv(filepath)
base_df.head()


In [ ]:
renamed_df = base_df.rename(columns={'Unnamed: 0': 'City ID', 'Max Temp': 'MaxTemp'})
renamed_df.head()

In [ ]:
renamed_df.MaxTemp.max()

In [ ]:
gmaps.configure(api_key=g_key)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
renamed_df.head()

In [ ]:
cleaned_df = renamed_df.dropna(subset=["Lat","Lng"])
cleaned_df

In [ ]:
# build gmap criteria 

In [ ]:
# build google maps heatmap criteria 

figure_layout = {
    'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

 

fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(
    cleaned_df[['Lat', 'Lng']], weights=cleaned_df['Humidity'],
    max_intensity=500, point_radius=10, dissipating = False
)
fig.add_layer(heatmap_layer)



# fig.add_layer(gmaps.traffic_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
cleaned_df.head()

In [ ]:
# add more filtering criteria for the very best vacation spots!

mask1 = (cleaned_df['MaxTemp'] < 80) & (cleaned_df['MaxTemp'] > 70)
mask2 = cleaned_df["Wind Speed"] < 10
mask3 = cleaned_df.Cloudiness < 10

df_sub = cleaned_df.loc[mask1 & mask2 & mask3].reset_index(drop=True)
df_sub = df_sub.dropna(how="any")
df_sub

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_locs = df_sub.loc[:, ["Lat", "Lng"]].values[0]
hotel_locs

In [ ]:
# utilize code from Google lesson and also group 4 project. 

target_coordinates = f"{hotel_locs[0]}, {hotel_locs[1]}"
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

response.url


In [ ]:
places_data = response.json()
print(json.dumps(places_data, indent=4, sort_keys=True))

In [ ]:
places_data["results"][0]

In [ ]:
names = []
addresses = []
location_codes = []
ratings = []

for _, row in df_sub.iterrows():
    # geocoordinates
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    print(response.url)
    # convert response to json
    places_data = response.json()
    
    try:
        name = places_data["results"][0]["name"]
        address = places_data["results"][0]["vicinity"]
        location_code = places_data["results"][0]["plus_code"]["compound_code"]
        rating = places_data["results"][0]["rating"]
    
        names.append(name)
        addresses.append(address)
        location_codes.append(location_code)
        ratings.append(rating)
    except:
        names.append("")
        addresses.append("")
        location_codes.append("")
        ratings.append(np.nan)
        
    time.sleep(1)
        
        
df_sub["hotel"] = names
df_sub["hotel_address"] = addresses
df_sub["hotel_location_code"] = location_codes
df_sub["rating"] = ratings

df_sub["hotel_full_address"] = df_sub["hotel_address"] + " " + df_sub["hotel_location_code"]

In [ ]:
df_sub

In [ ]:
hotel_df = df_sub.dropna()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel}</dd>
<dt>Name</dt><dd>{rating}</dd>
<dt>Name</dt><dd>{hotel_full_address}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(
    cleaned_df[['Lat', 'Lng']], weights=cleaned_df['Humidity'],
    max_intensity=500, point_radius=10, dissipating = False
)
fig.add_layer(heatmap_layer)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig
# Display figure
